In [1]:
#Import libraries
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import seaborn as sns
from scipy import stats
import math
import os
import random
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge, Lasso
from sklearn.datasets import make_regression
from sklearn.model_selection import learning_curve
from sklearn.metrics import r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
from sklearn.ensemble import RandomForestClassifier

# 5.6 Random Forest - solution

## 5.6.1 A little bit of theory

An often used technique in machine learning are random forests. Basically they are [decision trees](https://en.wikipedia.org/wiki/Decision_tree_learning), or in programmers terms, if-then-else structures, like the one shown below.

<img src="figures/tree.png" width=70%>

Decision trees are know to over-fit a lot because they just learn the train set by heart and store it. Random forests on the other hand combine multiple different (randomly initialized) decision trees that all over-fit in their own way. But by combining their output using a voting mechanism, they tend to cancel out eachothers mistakes. This approach is called an [ensemble](https://en.wikipedia.org/wiki/Ensemble_learning) and can be used for any combination of machine learning techniques. A schematical representation of how such a random forest works is shown below.

<img src="figures/random_forest.jpg">

Now let's try to use a random forest to solve the churn problem. (see [`sklearn.ensemble.RandomForestClassifier()`](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html))

**Some random forest parameters you can set to avoid overfitting:**
- n_estimators: number of trees you will grow
- max_depth: max number of splits each tree can make
- max_features: max features to consider when making a tree split
- min_samples_split: determines while building the tree how many data points you need at least before you create another split (this is an additional if-else structure)
- min_samples_leaf: sets the minimum amount of data points you have in each leaf

## 5.6.2 Importing data + making train/test data

We will try to predict if a customer of a firm will churn at the hand of several variables

In [ ]:
#make objects
df = pd.read_csv("data/churn.csv")
df = df.drop(["Phone", "Area Code", "State"], axis=1)
X = df.drop(["Churn"], axis=1)
y = df['Churn'].values.tolist()

# Split the data into test and training (30% for test)
X_train, X_test, Y_train, Y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.3, 
                                                    random_state = 123)

In [ ]:
#distribution churners and non churners
df.Churn.value_counts()

## 5.6.3 Model building

In [ ]:
rf = RandomForestClassifier(n_estimators=100)

# Train the classifier using the train data
rf = rf.fit(X_train, Y_train)

# Validate the classifier
accuracy = rf.score(X_test, Y_test)
print('Accuracy on test set: ' + str(accuracy))

# Make a confusion matrix
prediction = rf.predict(X_test)

conf_matrix = pd.DataFrame(
    confusion_matrix(Y_test, prediction), 
    columns=["Predicted False", "Predicted True"], 
    index=["Actual False", "Actual True"]
)

conf_matrix

In [ ]:
#calculate probabilities that customer will churn for test set
probs = rf.predict_proba(X_test)

fpr, tpr, threshold = roc_curve(Y_test, probs[:,1])

#calculate area under curve
roc_auc = auc(fpr,tpr)

plt.figure()
lw = 2
plt.figure(figsize=(10,10))
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.ylabel('True Positive Rate (sensitivity / recall)')
plt.xlabel('False Positive Rate (1- specificity)')
plt.title('Receiver operating characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()

## 5.6.4 Variable importance

Calculate the mean decrease in node impurity  / error rate from splitting on 1 variable
- Classification: decrease in Gini index or accuracy
- Regression: decrease in RSS or MSE

In [ ]:
# We'll use this library to make the display pretty
# install this module using: 
# $ conda install tabulate
from tabulate import tabulate

headers = ["name", "score"]
values = sorted(zip(X_train.columns, rf.feature_importances_), key=lambda x: x[1] * -1)
print(tabulate(values, headers, tablefmt="plain"))

In [ ]:
print(values)

## 5.6.5 Task 4: Hiring revisited

- Redo the previous excercise, using random forests.
- Compare your random forest models with the logistic regression models.

## 5.6.6 Solution

In [2]:
#make dataframe

hired_df = pd.read_csv('data/hired.csv')

#check head
hired_df.head()

,hired,education_level,experience,gender
0,0,6,2,0
1,0,4,0,1
2,1,6,6,1
3,1,6,3,1
4,0,4,1,0


In [3]:
#binary classification problem
hired_df.gender.value_counts()

0    45
1    39
Name: gender, dtype: int64

In [4]:
X = hired_df.drop(["hired"], axis=1)
y = hired_df['hired'].values.tolist()

# Split the data into test and training (30% for test)
X_train, X_test, Y_train, Y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.3, 
                                                    random_state = 123)

In [5]:
rf = RandomForestClassifier(n_estimators=100)

# Train the classifier using the train data
rf = rf.fit(X_train, Y_train)

# Validate the classifier
accuracy = rf.score(X_test, Y_test)
print('Accuracy on test set: ' + str(accuracy))

# Make a confusion matrix
prediction = rf.predict(X_test)

conf_matrix = pd.DataFrame(
    confusion_matrix(Y_test, prediction), 
    columns=["Predicted False", "Predicted True"], 
    index=["Actual False", "Actual True"]
)

conf_matrix

Accuracy on test set: 0.9615384615384616


,Predicted False,Predicted True
Actual False,16,0
Actual True,1,9


### Try Cross Validation to see if accuracy holds over different folds on dataset.

In [6]:
from sklearn.model_selection import cross_val_score

In [7]:
acc_cv = cross_val_score(rf, X, y, cv=5)

In [8]:
acc_cv

array([0.94444444, 0.94444444, 0.9375    , 0.9375    , 0.9375    ])

In [9]:
np.mean(acc_cv)

0.9402777777777779